# Promela
* https://spinroot.com/spin/Man/promela.html

# Grammar


```BNF
spec	: module [ module ] *

module	: proctype	                    /* proctype declaration */
	| init		                            /* init process       */
	| never		                            /* never claim        */
	| trace		                            /* event trace        */
	| utype		                            /* user defined types */
	| mtype		                            /* mtype declaration  */
	| decl_lst	                          /* global vars, chans */

proctype: [ active ] PROCTYPE name '(' [ decl_lst ]')'
	  [ priority ] [ enabler ] '{' sequence '}'

init	: INIT [ priority ] '{' sequence '}'

never	: NEVER	'{' sequence '}'

trace	: TRACE '{' sequence '}'

utype	: TYPEDEF name '{' decl_lst '}'

mtype	: MTYPE [ '=' ] '{' name [ ',' name ] * '}'

decl_lst: one_decl [ ';' one_decl ] *

one_decl: [ visible ] typename  ivar [',' ivar ] *
	| [ visible ] unsigned_decl

unsigned_decl: UNSIGNED name ':' const [ '=' any_expr ]

typename: BIT | BOOL | BYTE | SHORT | INT | MTYPE | CHAN
	| uname	                              /* user defined type names (see utype) */

active  : ACTIVE [ '[' const ']' ]	    /* instantiation */

priority: PRIORITY const	              /* simulation priority */

enabler : PROVIDED '(' expr ')'	        /* execution constraint */

visible	: HIDDEN | SHOW

sequence: step [ ';' step ] *

step    : stmnt	[ UNLESS stmnt ]
	| decl_lst
	| XR varref [',' varref ] *
	| XS varref [',' varref ] *

ivar    : name [ '[' const ']' ] [ '=' any_expr | '=' ch_init ]

ch_init : '[' const ']' OF '{' typename [ ',' typename ] * '}'

varref	: name [ '[' any_expr ']' ] [ '.' varref ]

send    : varref '!' send_args		      /* normal fifo send */
	| varref '!' '!' send_args	          /* sorted send */

receive : varref '?' recv_args		      /* normal receive */
	| varref '?' '?' recv_args	          /* random receive */
	| varref '?' '<' recv_args '>'	      /* poll with side-effect */
	| varref '?' '?' '<' recv_args '>'	  /* ditto */

poll    : varref '?' '[' recv_args ']'	/* poll without side-effect */
	| varref '?' '?' '[' recv_args ']'	  /* ditto */

send_args: arg_lst | any_expr '(' arg_lst ')'

arg_lst  : any_expr [ ',' any_expr ] *

recv_args: recv_arg [ ',' recv_arg ] *  |  recv_arg '(' recv_args ')'

recv_arg : varref | EVAL '(' varref ')' | [ '-' ] const

assign  : varref '=' any_expr	          /* standard assignment */
	| varref '+' '+'	                    /* increment */
	| varref '-' '-'	                    /* decrement */

stmnt	: IF options FI		                /* selection */
	| DO options OD		                    /* iteration */
	| FOR '(' range ')' '{' sequence '}'	/* iteration */
	| ATOMIC '{' sequence '}'	            /* atomic sequence */
	| D_STEP '{' sequence '}'	            /* deterministic atomic */
	| SELECT '(' range ')'	              /* non-deterministic value selection */
	| '{' sequence '}'	                  /* normal sequence */
	| send
	| receive
	| assign
	| ELSE			                          /* used inside options */
	| BREAK			                          /* used inside iterations */
	| GOTO name
	| name ':' stmnt	                    /* labeled statement */
	| PRINT '(' string [ ',' arg_lst ] ')'
	| ASSERT expr    
	| expr			                          /* condition */
	| c_code '{' ... '}'	                /* embedded C code */
	| c_expr '{' ... '}'
	| c_decl '{' ... '}'
	| c_track '{' ... '}'
	| c_state '{' ... '}'

range	: name ':' any_expr '..' any_expr
	| name IN name

options : ':' ':' sequence [ ':' ':' sequence ] *

andor	: '&' '&' | '|' '|'

binarop	: '+' | '-' | '*' | '/' | '%' | '&' | '^' | '|'
	| '>' | '<' | '>' '=' | '<' '=' | '=' '=' | '!' '='
	| '<' '<' | '>' '>' | andor

unarop	: '~' | '-' | '!'

any_expr: '(' any_expr ')'
	| any_expr binarop any_expr
	| unarop any_expr
	| '(' any_expr '-' '>' any_expr ':' any_expr ')'
	| LEN '(' varref ')'	                /* nr of messages in chan */
	| poll
	| varref
	| const 
	| TIMEOUT
	| NP_			/* non-progress system state */
	| ENABLED '(' any_expr ')'		        /* refers to a pid */
	| PC_VALUE '(' any_expr ')'		        /* refers to a pid */
	| name '[' any_expr ']' '@' name	    /* refers to a pid */
	| RUN name '(' [ arg_lst ] ')' [ priority ]
	| get_priority( expr )			          /* expr refers to a pid */
	| set_priority( expr , expr )		      /* first expr refers to a pid */

expr	: any_expr
	| '(' expr ')'
	| expr andor expr
	| chanpoll '(' varref ')'	            /* may not be negated */

chanpoll: FULL | EMPTY | NFULL | NEMPTY

string	: '"' [ any_ascii_char ] * '"'

uname	: name

name	: alpha [ alpha | number ] *

const	: TRUE | FALSE | SKIP | number [ number ] *

alpha	: 'a' | 'b' | 'c' | 'd' | 'e' | 'f' | 'g' | 'h' | 'i' | 'j'
	| 'k' | 'l' | 'm' | 'n' | 'o' | 'p' | 'q' | 'r' | 's' | 't'
	| 'u' | 'v' | 'w' | 'x' | 'y' | 'z'
	| 'A' | 'B' | 'C' | 'D' | 'E' | 'F' | 'G' | 'H' | 'I' | 'J'
	| 'K' | 'L' | 'M' | 'N' | 'O' | 'P' | 'Q' | 'R' | 'S' | 'T'
	| 'U' | 'V' | 'W' | 'X' | 'Y' | 'Z'
	| '_'

number	: '0' | '1' | '2' | '3' | '4' | '5' | '6' | '7' | '8' | '9'
```

# MetaTerms


## comments

```
/'*' [ any_ascii_char ]* '*'/
```

## false

The keyword false is a synonym of the constant value zero (0), and can be used in any context.

## for

The `for` statement was added in Spin Version 6 to simplify writing common loops over a user-defined range of values, the elements of an array, or the messages currently stored in a channel.

```
for '(' name ':' expr '..' expr ')' '{' sequence '}'
for '(' name in array ')' '{' sequence '}'
for '(' name in channel ')' '{' sequence '}'
```

## inline

```
inline name ( [ arg_lst ] ) { sequence }
```

a stylized version of a macro.

## ltl

Grammar:

```
	ltl ::= opd | ( ltl ) | ltl binop ltl | unop ltl
```

Operands (`opd`):
	true, false, user-defined names starting with a lower-case letter,
	or embedded expressions inside curly braces, e.g.,: `{ a+b>n }`.

Unary Operators (`unop`):

-	`[]`	(the temporal operator `always`)
-	`<>`	(the temporal operator `eventually`)
-	`!` 	(the boolean operator for `negation`)

Binary Operators (`binop`):

-	`U` 	(the temporal operator `strong until`)
-	`W`	(the temporal operator `weak until` (only when used in inline formula)
-	`V` 	(the dual of `U`): `(p V q)` means `!(!p U !q))`
-	`&&`	(the boolean operator for `logical and`)
-	`||`	(the boolean operator for `logical or`)
-	`/\`	(alternative form of `&&`)
-	`\/`	(alternative form of `||`)
-	`->`	(the boolean operator for `logical implication`)
-	`<->`	(the boolean operator for `logical equivalence`)

**Inline Specification**: The easiest way to specify an LTL property is to specify it inline. The formula is specified globally (i.e., outside all proctype or init declarations) with the following syntax:

```
	ltl [ name ] '{'  formula '}'
```

**Alternative Method**: Spin can translate LTL formulae into Promela never claims with command line option `-f`.

## macros

```
#define name	token-string
#define name(arg, ..., arg)	token-string
#ifdef name
#ifndef name
#if constant-expression
#else
#endif
#undef name
#include "filename"
```

Promela source text is always processed by the C preprocessor, conventionally named cpp , before being parsed by Spin.

## select

```
select '(' name ':' expr '..' expr ')'
```

The `select` statement was added in Spin Version 6 to simplify writing a standard selection of a non-deterministic value from a specified range of values.

## skip

The keyword skip is a meta term that is translated by the Spin lexical analyzer into the constant value one (1), just like the predefined boolean constant true . The intended use of the shorthand is stand-alone, as a dummy statement. When used as a statement, the skip is interpreted as a special case of a condition statement. This condition statement is always executable, and has no effect when executed, other than a possible change of the control-state of the executing process.

## true

The keyword true is a synonym of the constant value one (1) , and can be used in any context.

# Declarators

## accept

```
accept[a-zA-Z0-9_]*: stmnt
```

Accept labels are used to formalize Buchi acceptance conditions. 
They are most often used inside never claims, but their special meaning is also recognized when they are used inside trace assertions, or in the body of a proctype declaration.


## active

```
active proctype name ( [ decl_lst ] ) { sequence }
active '[' const ']' proctype name ( [ decl_lst ] ) { sequence }
```

The keyword `active` can be prefixed to any proctype declaration to define a set of processes that are required to be active (i.e., running) in the initial system state.


## arrays


``` 
typename name '[' const ']' [ = any_expr ]
```

An object of any predefined or user-defined datatype can be declared either as a scalar or as an array. The array elements are distinguished from one another by their array index.




## chan
## datatypes
## end
## hidden
## init
## local
## mtype
## never
## notrace
## priority
## proctype
## progress
## provided
## show
## trace
## typedef
## xr
## xs

# Control Flow

- atomic: 原子
- break: 结束循环
- d_step
- do: 重复
- for
- goto: 跳转
- if: 选择
- labels
- separators
- sequence
- unless

# Basic Statements

- assert
- assign
- condition
- printf
- receive
- send

# Predefined

- `_`: 全局只写的变量
- `_last`: 执行最后一个步骤的进程号
- `_pid`: 每个进程的标识符
- `_nr_pr`
- cond_expr
- else
- empty
- `enabled(pid)`: 进程当前状态是否至少有一个语句可执行. (只用于never声明)
- eval
- full
- len
- nempty
- nfull
- `np_`: 当前系统状态是否实在可进展状态
- operators
- `pc_value(pid)`: 返回进程的当前控制状态. (只用于never声明)
- poll
- remoterefs
- run
- stdin
- `timeout`: 当且仅当系统中没有其他语句是可执行的时为true - TSMC P.57
- `procname[pid]@label`: 进程中下一个可执行的语句是否是进程类型`procname`中标记`label`标签的语句. (只用于never声明)


# Embedded C Code

- c_code
- c_decl
- c_expr
- c_state
- c_track


# Omissions

- float
- hierarchy
- pointers
- probabilities
- procedures
- rand
- realtime
- scanf